# Collocation Optimal Control for a Robot Manipulator
Objective: optimize a trajectory for a robot manipulator using the collocation approach.

# Modeling a Robot Manipulator
We use as state of the robot manipulator its joint positions and velocities:
$$ x = (q, \dot{q})$$
The control inputs would normally be the joint torques $\tau$:
$$ u = \tau $$

The system dynamics is then represented by the so-called **forward dynamics** function:
$$ \ddot{q} = M(q)^{-1}(u - h(x))$$

The joint torque bounds are then simple bounds on the control inputs:
$$ \tau_{min} \le u \le \tau_{max}$$

## Alternative model
Rather than using the torques as control input, we can assume the control inputs are the joint accelerations $\ddot{q}$.

The system dynamics then is simply a double integrator:
$$ \ddot{q} = u$$

However, the torque bounds become nonlinear constraints depending on both $x$ and $u$, because the joint torques need to be computed using the **inverse dynamics** function:
$$\tau_{min} \le \underbrace{M(q) u + h(x)}_{\text{inv_dyn}(q, \dot{q}, \ddot{q})} \le \tau_{max}$$

# Model Choice
We can choose whether to adopt the first model:
* $u = \tau$
* Nonlinear dynamics (forward dynamics)
* Simple bounds for torque limits

Or we can use the second model:
* $u = \ddot{q}$
* Linear dynamics
* Nonlinear inequalities for torque limits (inverse dynamics)

Since the **inverse dynamics** function is faster to evaluate than the **forward dynamics** function, the second model ($u=\ddot{q}$) should be computationally more efficient.

Moreover, the library (Adam) that we use for modeling the robot dynamics does not currently implement the **forward dynamics** function directly, so it is easier to use the second model.

# Problem Formulation
Let's implement the following optimal control problem:
$$ \min_{X, U} \quad \sum_{k=0}^{N-1} (w_p ||y(q_k)-y^d||^2 + w_v ||\dot{q}_k||^2 + w_a ||u_k||^2) + w_{final} ||y(q_N)-y^d||^2 + w_{final} ||\dot{q}||^2$$
$$\text{subject to} \quad 
q_{k+1} = q_k + \Delta t \dot{q}_k, \qquad 
\dot{q}_{k+1} = \dot{q}_k + \Delta t u_k \qquad 
\forall k \in [0, N-1] \qquad \qquad \qquad \qquad$$
$$\qquad \qquad 
q^{min} \le q_{k} \le q^{max}, \qquad 
-\dot{q}^{max} \le \dot{q}_{k} \le \dot{q}^{max} \quad 
\forall k \in [1, N]  \qquad \qquad \qquad \qquad$$
$$\qquad \qquad 
\tau_{min} \le \text{inv_dyn}(q_k, \dot{q}_k, u_k) \le \tau_{max} \qquad\qquad\qquad
\forall k \in [0, N-1]  \qquad \qquad \qquad \qquad$$
$$ x_0 = x_{init} \qquad \qquad  \qquad \qquad \qquad \qquad$$
where $y(q)$ is the position of the end-effector.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from adam.casadi.computations import KinDynComputations
import casadi as cs
from time import time as clock
from time import sleep

import orc.utils.plot_utils as plut
from example_robot_data.robots_loader import load
import orc.optimal_control.casadi_adam.conf_ur5 as conf_ur5

In [2]:
robot = load("ur5")
joints_name_list = [s for s in robot.model.names[1:]] # skip the first name because it is "universe"
nq = len(joints_name_list)  # number of joints
nx = 2*nq                   # size of the state variable

In [3]:
dt = 0.02           # OCP time step
N = 75              # number of OCP time steps
q0 = np.zeros(nq)   # initial joint configuration
dq0= np.zeros(nq)   # initial joint velocities
x_init = np.concatenate([q0, dq0])
frame_name = "ee_link"
ee_des = np.array([0, -0.45, 0]) # desired end-effector position

In [4]:
# create the dynamics function
q   = cs.SX.sym('q', nq)
dq  = cs.SX.sym('dq', nq)
ddq = cs.SX.sym('ddq', nq)
state = cs.vertcat(q, dq)
rhs    = cs.vertcat(dq, ddq)
f = cs.Function('f', [state, ddq], [rhs])

In [5]:
# create a Casadi inverse dynamics function
kinDyn = KinDynComputations(robot.urdf, joints_name_list)
H_b = cs.SX.eye(4)     # base configuration
v_b = cs.SX.zeros(6)   # base velocity
bias_forces = kinDyn.bias_force_fun()
mass_matrix = kinDyn.mass_matrix_fun()
# discard the first 6 elements because they are associated to the robot base
h = bias_forces(H_b, q, v_b, dq)[6:]
M = mass_matrix(H_b, q)[6:,6:]
tau = M @ ddq + h
inv_dyn = cs.Function('inv_dyn', [state, ddq], [tau])

In [6]:
# create a Casadi forward kinematics function
if(frame_name not in kinDyn.rbdalgos.model.links.keys()):
    print("ERROR. Frame name can only take values from this list")
fk_fun = kinDyn.forward_kinematics_fun(frame_name)
ee_pos = fk_fun(H_b, q)[:3, 3]
fk = cs.Function('fk', [q], [ee_pos])

In [7]:
# pre-compute state and torque bounds
lbx = robot.model.lowerPositionLimit.tolist() + (-robot.model.velocityLimit).tolist()
ubx = robot.model.upperPositionLimit.tolist() + robot.model.velocityLimit.tolist()
tau_min = (-robot.model.effortLimit).tolist()
tau_max = robot.model.effortLimit.tolist()

In [8]:
# create all the decision variables
def create_decision_variables(N, nx, nu, lbx, ubx):
    opti = cs.Opti()
    X, U = [], []
    for k in range(N+1): 
        X += [opti.variable(nx)]
        opti.subject_to( opti.bounded(lbx, X[-1], ubx) )
    for k in range(N): 
        U += [opti.variable(nu)]
    return opti, X, U

In [9]:
def define_running_cost_and_dynamics(opti, X, U, N, dt, x_init, ee_des, w_p, w_v, w_a, tau_min, tau_max):
    opti.subject_to(X[0] == x_init)
    cost = 0.0
    for k in range(N):     
        # Compute cost function
        ee_pos = fk(X[k][:nq])
        cost += w_p * (ee_pos - ee_des).T @ (ee_pos - ee_des) * dt 
        cost += w_v * X[k][nq:].T @ X[k][nq:] * dt
        cost += w_a * U[k].T @ U[k] * dt

        # Add dynamics constraints
        opti.subject_to(X[k+1] == X[k] + dt * f(X[k], U[k]))

        # Add torque constraints
        opti.subject_to( opti.bounded(tau_min, inv_dyn(X[k], U[k]), tau_max))
    return cost

In [10]:
# add the terminal cost on end-effector position and velocity
def define_terminal_cost(X, ee_des, w_final):
    err_pos = fk(X[-1][:nq]) - ee_des
    cost = w_final * err_pos.T @ err_pos
    cost += w_final * X[-1][nq:].T @ X[-1][nq:]
    return cost

In [21]:
def create_and_solve_ocp(N, nx, nq, lbx, ubx, dt, x_init, 
                        ee_des, w_p, w_v, w_a, w_final, tau_min, tau_max):
    opti, X, U = create_decision_variables(N, nx, nq, lbx, ubx)
    running_cost = define_running_cost_and_dynamics(opti, X, U, N, dt, x_init, 
                                                    ee_des, w_p, w_v, w_a, tau_min, tau_max)
    terminal_cost = define_terminal_cost(X, ee_des, w_final)
    opti.minimize(running_cost + terminal_cost)
    
    # Create the optimization problem
    opts = {"ipopt.print_level": 0, "print_time": 0, "ipopt.tol": 1e-4,
            "ipopt.constr_viol_tol": 1e-4, "ipopt.compl_inf_tol": 1e-4}
    opti.solver("ipopt", opts)
    
    t0 = time.time()
    sol = opti.solve()
    t1 = time.time()
    print("Solver time", t1-t0)
    
    return sol, X, U

In [12]:
def extract_solution(sol, X, U):
    x_sol = np.array([sol.value(X[k]) for k in range(N+1)]).T
    ddq_sol = np.array([sol.value(U[k]) for k in range(N)]).T
    q_sol = x_sol[:nq,:]
    dq_sol = x_sol[nq:,:]
    # compute joint torques trajectory
    tau = np.zeros((nq, N))
    for i in range(N):
        tau[:,i] = inv_dyn(x_sol[:,i], ddq_sol[:,i]).toarray().squeeze()
    # compute end-effector trajectory
    ee = np.zeros((3, N+1))
    for i in range(N+1):
        ee[:,i] = fk(x_sol[:nq,i]).toarray().squeeze()
    return q_sol, dq_sol, ddq_sol, tau, ee

In [13]:
from orc.utils.robot_wrapper import RobotWrapper
from orc.utils.robot_simulator import RobotSimulator
from orc.utils.viz_utils import addViewerSphere, applyViewerConfiguration
import time

r = RobotWrapper(robot.model, robot.collision_model, robot.visual_model)
simu = RobotSimulator(conf_ur5, r)
simu.init(q0, dq0)
simu.display(q0)

REF_SPHERE_RADIUS = 0.05
EE_REF_SPHERE_COLOR = np.array([1, 0, 0, .5])
addViewerSphere(r.viz, 'world/ee_ref', REF_SPHERE_RADIUS, EE_REF_SPHERE_COLOR)

def display_motion(q_traj):
    applyViewerConfiguration(r.viz, 'world/ee_ref', ee_des.tolist()+[0,0,0,1.])
    for i in range(N):
        t0 = time.time()
        simu.display(q_traj[:,i])
        t1 = time.time()
        if(t1-t0 < dt):
            sleep(dt - (t1-t0))

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7003/static/


In [14]:
def solve_and_display(log_w_p, log_w_v, log_w_a, log_w_final):
    sol, X, U = create_and_solve_ocp(N, nx, nq, lbx, ubx, dt, x_init, ee_des, 
                                     10**log_w_p, 10**log_w_v, 10**log_w_a, 10**log_w_final, 
                                     tau_min, tau_max)
    q_sol, dq_sol, u_sol, tau, ee = extract_solution(sol, X, U)
    display_motion(q_sol)

In [15]:
r.viz.viewer.jupyter_cell()

In [22]:
from ipywidgets import interact, fixed
interact(solve_and_display, log_w_p=(-6,0,1), log_w_v=(-6,0,1), log_w_a=(-6,0,1), log_w_final=(-6,0,1))

interactive(children=(IntSlider(value=-3, description='log_w_p', max=0, min=-6), IntSlider(value=-3, descripti…

<function __main__.solve_and_display(log_w_p, log_w_v, log_w_a, log_w_final)>

In [19]:
def solve_and_plot(log_w_p, log_w_v, log_w_a, log_w_final):
    sol, X, U = create_and_solve_ocp(N, nx, nq, lbx, ubx, dt, x_init, ee_des, 
                                     10**log_w_p, 10**log_w_v, 10**log_w_a, 10**log_w_final, 
                                     tau_min, tau_max)
    q_sol, dq_sol, u_sol, tau, ee = extract_solution(sol, X, U)
    tt = np.linspace(0, (N+1)*dt, N+1)

    plt.figure(figsize=(10, 4))
    for i in range(3):
        plt.plot([tt[0], tt[-1]], [ee_des[i], ee_des[i]], ':', label=f'EE des {i}', alpha=0.7)
        plt.plot(tt, ee[i,:].T, label=f'EE {i}', alpha=0.7)
    plt.xlabel('Time [s]')
    plt.ylabel('End-effector pos [m]')
    plt.legend()
    plt.grid(True)

    plt.figure(figsize=(10, 4))
    for i in range(dq_sol.shape[0]):
        plt.plot(tt, dq_sol[i,:].T, label=f'dq {i}', alpha=0.7)
    plt.xlabel('Time [s]')
    plt.ylabel('Joint velocity [rad/s]')
    plt.legend()
    plt.grid(True)

    plt.figure(figsize=(10, 4))
    for i in range(tau.shape[0]):
        plt.plot(tt[:-1], tau[i,:].T, label=f'tau {i}', alpha=0.7)
    plt.xlabel('Time [s]')
    plt.ylabel('Joint torque [Nm]')
    plt.legend()
    plt.grid(True)
    plt.show()

In [20]:
interact(solve_and_plot, log_w_p=(-6,0,1), log_w_v=(-6,0,1), log_w_a=(-6,0,1), log_w_final=(-6,0,1))

interactive(children=(IntSlider(value=-3, description='log_w_p', max=0, min=-6), IntSlider(value=-3, descripti…

<function __main__.solve_and_plot(log_w_p, log_w_v, log_w_a, log_w_final)>